In [2]:
# código para criar a leitura dos dados armazenados em csv
import configparser
import itertools
import os
from datetime import datetime
import pandas as pd

In [20]:
# Carregar o arquivo de configuração
config = configparser.ConfigParser()
config.read("config.ini")

transformacao_dir = config["Paths"]["transformacao_dir"]

In [45]:

def process_csv(file_path):
    """Função para processar o arquivo CSV e retornar um DataFrame"""
    # Ler o arquivo CSV, extraindo as 10 primeiras linhas (para pegar a linha 4)
    temp_df = pd.read_csv(file_path, nrows=10, header=None, encoding="ISO-8859-1")

    # Extrair o mês da linha 4 (index 3 porque indexação começa em 0)
    linha = (
        temp_df.iloc[3].dropna().values[0]
    )  # Supondo que o mês esteja na primeira coluna não vazia

    # Extrair o mês da linha
    mes = linha.split(" ")[1].split("/")[0]
    ano = linha.split(" ")[1].split("/")[1]

    # Ler o arquivo inteiro e remover as 7 primeiras e 4 últimas linhas
    df = pd.read_csv(
        file_path, skiprows=7, encoding="ISO-8859-1", sep=";"
    )  # Ignorar as 7 primeiras linhas
    df = df[:-2]  # Remover as 2 últimas linhas

    # Renomear e preencher a última coluna para mes
    df.rename(columns={df.columns[-1]: "Mes"}, inplace=True)
    df["Mes"] = mes

    # Adicionar a coluna de ano
    df["Ano"] = ano

    return df


def concat_csv_files(files):
    """Função para concatenar os arquivos CSV em grupos de 100"""
    # Inicializar um DataFrame vazio
    df = pd.DataFrame()

    # Colunas esperadas no DataFrame
    colunas =['Uf',
            'Ibge',
            'Municipio',
            'Asma',
            'Desnutrição',
            'Diabetes',
            'DPOC',
            'Hipertensão arterial',
            'Obesidade',
            'Pré-natal',
            'Puericultura',
            'Puerpério (até 42 dias)',
            'Saúde sexual e reprodutiva',
            'Tabagismo',
            'Usuário de álcool',
            'Usuário de outras drogas',
            'Saúde mental',
            'Reabilitação',
            'D.Transmissíveis - Dengue',
            'Doenças transmissíveis - DST',
            'D.Transmissíveis - Hanseníase',
            'D.Transmissíveis - Tuberculose',
            'Rast. câncer de mama',
            'Rast. câncer do colo do útero',
            'Rast. risco cardiovascular',
            'Mes',
            'Ano']
    # Iterar sobre os arquivos
    for i, file in enumerate(files):
        # Processar o arquivo
        temp_df = process_csv(file)
        # Pular arquivos que não possuem as colunas esperadas
        if temp_df.columns.tolist() != colunas:
            print(file, "não possui as colunas esperadas")
            print("Colunas DF", temp_df.columns.tolist())
            print()
            continue
        # Concatenar com o DataFrame principal
        df = pd.concat([df, temp_df])

        # Salvar a cada 100 arquivos
        if i % 100 == 0 and i > 0:
            df.to_csv(f"partial_{i}.csv", index=False)
            df = pd.DataFrame()
    try:
        # Salvar o restante
        df.to_csv(f"partial_{i}.csv", index=False)
    # Catch UnboundLocalError
    except UnboundLocalError:
        print("Menos de 100 arquivos foram processados")
        df.to_csv(f"partial.csv", index=False)


# Função para listar os arquivos CSV
def list_files(directory="/home/daniel/Downloads"):
    """Função para listar os arquivos CSV em um diretório"""
    import os

    # Listar os arquivos com o diretório
    files = [
        os.path.join(directory, file)
        for file in os.listdir(directory)
        if file.endswith(".csv")
    ]

    return files


def concat_final_csv(name, dir="."):    
    """Função para concatenar os arquivos CSV"""
    # Inicializar um DataFrame vazio
    df = pd.DataFrame()
    for file in list_files(dir):
        print(file)
        df_temp = pd.read_csv(file)
        print(df_temp.head())
        df = pd.concat([df, df_temp])
    # Remove duplicados
    print(df.shape)
    df.drop_duplicates(inplace=True)
    print(df.shape)
    # Salvar o arquivo final
    df.to_csv(f"{name}.csv", index=False)
    return df


def ausentes(df):
    """Função para contar os valores ausentes em um DataFrame"""

    now = datetime.now()
    ano_atual = now.year + 1
    mes_atual = now.month + 1

    anos = list(range(2021, ano_atual))
    mes = list(range(1, mes_atual))

    combinacoes = list(itertools.product(df['Uf'].unique(), df['Mes'].unique(), anos))
    combinacoes_24 = list(itertools.product(df['Uf'].unique(), mes, [2024]))

    combinacoes = combinacoes + combinacoes_24

    # Filtrar as combinações que já existem no DataFrame
    combinacoes_existentes = set(zip(df['Uf'], df['Mes'], df['Ano']))

    # Identificar as combinações ausentes
    combinacoes_ausentes = [comb for comb in combinacoes if comb not in combinacoes_existentes]
    return combinacoes_ausentes


def existentes(df):
    """Função para contar os valores existentes em um DataFrame"""

    # Filtrar as combinações que já existem no DataFrame
    combinacoes_existentes = set(zip(df['Uf'], df['Mes'], df['Ano']))

    return combinacoes_existentes


def atualiza_controle(combinacoes_existentes):
    """Função para atualizar o controle de combinações"""
    with open('controle/producao.txt', 'w') as f:
        for comb in combinacoes_existentes:
            if comb[1] < 10:
                f.write(f'{comb[0]}_0{comb[1]}/{comb[2]}\n')
            else:
                f.write(f'{comb[0]}_{comb[1]}/{comb[2]}\n')


def remove_temp_files(transformacao_dir):
    """Função para remover os arquivos temporários"""
    for file in os.listdir("."):
        if file.startswith("partial_"):
            os.remove(file)
    for file in os.listdir(transformacao_dir):
        if file.endswith(".csv"):
            os.remove(os.path.join(transformacao_dir, file))



if __name__ == "__main__":
    print("Iniciando a transformação dos dados")
    # Listar os arquivos
    files = list_files(transformacao_dir)
    print("Concatenando os arquivos")
    # Concatenar os arquivos
    concat_csv_files(files)
    print("Concatenando o arquivo final")
    # Concatenar o arquivo final
    df = concat_final_csv('procedimentos')
    print("Removendo os arquivos temporários")
    if len(df) > 1000:
        # remove_temp_files(transformacao_dir)
        print("Transformação concluída")
    else:
        print("Erro na transformação")

Iniciando a transformação dos dados
Concatenando os arquivos
Concatenando o arquivo final
./partial_100.csv
   Uf      Ibge            Municipio   Asma  Desnutrição  Diabetes  DPOC  \
0  PI  220900.0  RIO GRANDE DO PIAUÍ    1.0          2.0       5.0   1.0   
1  PR  410520.0           CERRO AZUL    2.0          2.0     145.0   2.0   
2  RJ  330190.0             ITABORAÍ   86.0         15.0     970.0  28.0   
3  RS  431410.0          PASSO FUNDO  161.0         52.0     594.0  51.0   
4  GO  520630.0       CRISTIANÓPOLIS    5.0          0.0      68.0   1.0   

   Hipertensão arterial  Obesidade  Pré-natal  ...  Reabilitação  \
0                25.000        2.0       32.0  ...         122.0   
1               280.000       39.0       49.0  ...         200.0   
2                 2.964      171.0      586.0  ...         358.0   
3                 1.211      269.0      853.0  ...           0.0   
4               168.000       15.0       11.0  ...           1.0   

   D.Transmissíveis - Deng

In [43]:
df = pd.read_csv('partial_100.csv')
print(len(df))
df2 = pd.read_csv('partial_134.csv')
print(len(df2))

df = pd.concat([df, df2])
df

477562
136542


,Uf,Ibge,Municipio,Asma,Desnutrição,Diabetes,DPOC,Hipertensão arterial,Obesidade,Pré-natal,...,Reabilitação,D.Transmissíveis - Dengue,Doenças transmissíveis - DST,D.Transmissíveis - Hanseníase,D.Transmissíveis - Tuberculose,Rast. câncer de mama,Rast. câncer do colo do útero,Rast. risco cardiovascular,Mes,Ano
0,PI,220900.0,RIO GRANDE DO PIAUÍ,1.0,2.0,5.0,1.0,25.000,2.0,32.0,...,122.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,MAI,2021.0
1,PR,410520.0,CERRO AZUL,2.0,2.0,145.0,2.0,280.000,39.0,49.0,...,200.0,0.0,0.0,0.0,0.0,19.0,20.0,15.0,MAI,2021.0
2,RJ,330190.0,ITABORAÍ,86.0,15.0,970.0,28.0,2.964,171.0,586.0,...,358.0,0.0,11.0,0.0,3.0,125.0,135.0,139.0,MAI,2021.0
3,RS,431410.0,PASSO FUNDO,161.0,52.0,594.0,51.0,1.211,269.0,853.0,...,0.0,1.0,52.0,0.0,31.0,6.0,36.0,0.0,MAI,2021.0
4,GO,520630.0,CRISTIANÓPOLIS,5.0,0.0,68.0,1.0,168.000,15.0,11.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MAI,2021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136537,MG,312310.0,DORES DE GUANHÃES,2.0,3.0,34.0,2.0,83.000,5.0,38.0,...,2.0,0.0,1.0,0.0,0.0,1.0,9.0,0.0,AGO,2019.0
136538,PR,412402.0,SANTA TEREZA DO OESTE,21.0,1.0,38.0,5.0,51.000,1.0,103.0,...,0.0,1.0,8.0,1.0,0.0,0.0,0.0,0.0,AGO,2019.0
136539,SP,350480.0,BÁLSAMO,11.0,2.0,48.0,15.0,59.000,54.0,55.0,...,0.0,10.0,3.0,1.0,0.0,0.0,90.0,0.0,AGO,2019.0
136540,BA,292670.0,RIO DE CONTAS,9.0,19.0,82.0,4.0,391.000,18.0,97.0,...,357.0,1.0,2.0,2.0,1.0,0.0,6.0,2.0,AGO,2019.0


In [29]:
print("Iniciando a transformação dos dados")
# Listar os arquivos
files = list_files(transformacao_dir)
print("Concatenando os arquivos")
# Concatenar os arquivos
concat_csv_files(files)

print("Concatenando o arquivo final")
# Concatenar o arquivo final
df = concat_final_csv('procedimentos')
"""
print("Gravando os arquivos existentes")
atualiza_controle(existentes(df))
print("Removendo os arquivos temporários")
remove_temp_files(transformacao_dir)
print("Transformação concluída")"""


Iniciando a transformação dos dados
Concatenando os arquivos
Concatenando o arquivo final


'\nprint("Gravando os arquivos existentes")\natualiza_controle(existentes(df))\nprint("Removendo os arquivos temporários")\nremove_temp_files(transformacao_dir)\nprint("Transformação concluída")'